# Analisis Exploratorio de Datos

## Importar datos, librerias y creación de funciones

In [ ]:
# Importar librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Cargar dataset redWine y whiteWine
redWine = pd.read_csv('winequality-red.csv', sep=';')
whiteWine = pd.read_csv('winequality-white.csv', sep=';')

In [ ]:
# Funcion para mostrar histograma y boxplot de un atributo
def histograma_boxplot(dataset, atributo):
  plt.figure(figsize=(10,4))

  plt.subplot(1,2,1)
  sns.histplot(dataset[atributo])

  plt.subplot(1,2,2)
  sns.boxplot(data=dataset[atributo])

In [ ]:
# Funcion deteccion de outliers
def deteccion_outliers(dataset):
  cols = dataset.columns
  for i in cols:
    if i == 'type':
      continue
    q1 = dataset[i].quantile(0.25)
    q3 = dataset[i].quantile(0.75)
    iqr = q3 - q1
    lim_inf = q1 - 1.5 * iqr
    lim_sup = q3 + 1.5 * iqr
    outliers = dataset[(dataset[i] < lim_inf) | (dataset[i] > lim_sup)].shape[0]
    print(f"El atributo {i} tiene {outliers} outliers")

In [ ]:
# Funcion resumen de datos
def resumen_datos(dataset):
  print("Cantidad de datos en wine: ", dataset.shape[0])
  print("Calidad media de vinos: ", dataset['quality'].mean())
  print("Cantidad de vinos con calidad mayor a la media: ", dataset[dataset['quality'] > dataset['quality'].mean()].shape[0])
  print("Cantidad de vinos con calidad menor a la media: ", dataset[dataset['quality'] < dataset['quality'].mean()].shape[0])
  print("La desviacion estandar de la calidad de los vinos es: ", dataset['quality'].std())
  print("La calidad minima de los vinos es: ", dataset['quality'].min())
  print("La calidad maxima de los vinos es: ", dataset['quality'].max())
  print("El 25% de los vinos tiene una calidad menor a: ", dataset['quality'].quantile(0.25))
  print("El 75% de los vinos tiene una calidad mayor a: ", dataset['quality'].quantile(0.75))
  print("El ultimo 25% de los vinos tiene una calidad entre ", dataset['quality'].quantile(0.25), " y ", dataset['quality'].quantile(0.75))
  print("La mayor concentracion de vinos se encuentra entre ", dataset['quality'].quantile(0.25), " y ", dataset['quality'].quantile(0.75))

In [ ]:
# Funcion detección de asimetria
def deteccion_asimetria(dataset):
  cols = dataset.columns
  for i in cols:
    if i == 'type':
      continue
    print(f"El atributo {i} tiene una asimetria de {dataset[i].skew()}")

In [ ]:
# Funcion mostrar mayor correlacion entre atributos
def mayor_correlacion(dataset, max_valor = 0, atributo = ''):
  corr = dataset.corr()
  if atributo != '':
    corr_quality = corr[[atributo]].copy()
    corr_quality['abs_corr'] = corr_quality[atributo].abs()
    corr_quality = corr_quality[corr_quality['abs_corr'] != 1]  # Excluir correlaciones igual a 1
    corr_quality = corr_quality.sort_values('abs_corr', ascending=False).head(3)

    print(corr_quality)
  elif max_valor != 0:
    max_valor = abs(max_valor)
    high_corr = corr[(corr > max_valor) | (corr < -max_valor)]
    high_corr = high_corr.stack().reset_index()
    high_corr['abs_corr'] = high_corr[0].abs()
    high_corr = high_corr[high_corr[0] != 1]  # Filtrar correlaciones diferentes de 1
    high_corr = high_corr.sort_values('abs_corr', ascending=False).drop_duplicates(0)
    print(high_corr)

## Analisis de los datos de Red Wine

### Resumen de Datos

In [ ]:
# Observar dataset redWine
redWine.head()

In [ ]:
# Cantidad de datos en redWine y whiteWine
print("Cantidad de datos en redWine: ", redWine.shape[0])

In [ ]:
# Tipos de datos en redWine
print("Red Wine:")
redWine.info()

Se observa que no es necesario hacer algun cambio en los datos, ya que no hay datos categoricos

In [ ]:
# Resumen de datos en relacion de RedWine en base al atributo 'quality'
resumen_datos(redWine)

### Graficos de Histogramas y Boxplots

In [ ]:
# Cantidades de vinos por calidad
sns.countplot(x='quality', data=redWine)

In [ ]:
# Por cada atributo, mostrar histograma y boxplot
for atributo in redWine.columns:
  histograma_boxplot(redWine, atributo)

### Deteccion de Outliers

In [ ]:
# Deteccion de outliers en wine
deteccion_outliers(redWine)

### Analisis Asimetrías

In [ ]:
# Detectar asimetria en redWine
deteccion_asimetria(redWine)

### Analisis de Correlaciones entre variables

In [ ]:
# Coeficiente de correlación de Pearson
plt.figure(figsize=(10,10))
sns.heatmap(redWine.corr(method='pearson'), annot=True, cmap='RdYlGn')
plt.show()

In [ ]:
# Atributos con mayor correlacion entre ellos
mayor_correlacion(redWine, max_valor=0.6)

### Analisis de Correlaciones en relacion a la variable Quality

In [ ]:
# Correlacion con la variable 'quality' del dataset wine a través de un mapa de calor
plt.figure(figsize=(5,5))
sns.heatmap(redWine.corr()[['quality']], annot=True, cmap='RdYlGn')
plt.show()

In [ ]:
# Atributos con mayor correlacion con 'quality'
mayor_correlacion(redWine, atributo='quality')

## Analisis de los datos de White Wine

### Resumen de Datos

In [ ]:
# Observar dataset WhiteWine
whiteWine.head()

In [ ]:
# Cantidad de datos en whiteWine
print("Cantidad de datos en redWine: ", whiteWine.shape[0])

In [ ]:
# Tipos de datos en WhiteWine
print("White Wine:")
whiteWine.info()

Se observa que no es necesario hacer algun cambio en los datos, ya que no hay datos categoricos

In [ ]:
# Resumen de datos en relacion de WhiteWine en base al atributo 'quality'
resumen_datos(whiteWine)

### Graficos de Histogramas y Boxplots

In [ ]:
# Cantidades de vinos por calidad
sns.countplot(x='quality', data=whiteWine)

In [ ]:
# Por cada atributo, mostrar histograma y boxplot
for atributo in whiteWine.columns:
  histograma_boxplot(whiteWine, atributo)

### Deteccion de Outliers

In [ ]:
# Deteccion de outliers en wine
deteccion_outliers(whiteWine)

### Analisis Asimetrías

In [ ]:
# Detectar asimetria en redWine
deteccion_asimetria(whiteWine)

### Analisis de Correlaciones entre variables

In [ ]:
# Coeficiente de correlación de Pearson
plt.figure(figsize=(10,10))
sns.heatmap(whiteWine.corr(method='pearson'), annot=True, cmap='RdYlGn')
plt.show()

In [ ]:
# Atributos con mayor correlacion entre ellos
mayor_correlacion(whiteWine, max_valor=0.6)

### Analisis de Correlaciones en relacion a la variable Quality

In [ ]:
# Correlacion con la variable 'quality' del dataset wine a través de un mapa de calor
plt.figure(figsize=(5,5))
sns.heatmap(whiteWine.corr()[['quality']], annot=True, cmap='RdYlGn')
plt.show()

In [ ]:
# Atributos con mayor correlacion con 'quality'
mayor_correlacion(whiteWine, atributo='quality')

# Procesamiento de datos

## Funciones 

In [ ]:
# Función de entrenamiento utilizando descenso de gradiente por lotes
def train_linear_regression(X, y, learning_rate, num_iterations, tipo='simple'):
    # Inicializa los parámetros del modelo
    if tipo == 'simple':
        theta0 = 0.0
        theta1 = 0.0
    elif tipo == 'multiple':
        theta = np.zeros(X.shape[1]+1)
        
    m = len(X)

    # Descenso de gradiente
    for _ in range(num_iterations):
        if tipo == 'simple':
            # Calcula las predicciones y el error
            y_pred = theta0 + theta1 * X
            error = y_pred - y

            # Actualiza los parámetros utilizando el gradiente
            theta0 -= (learning_rate / m) * np.sum(error)
            theta1 -= (learning_rate / m) * np.sum(error * X)
        elif tipo == 'multiple':
            # Calcula las predicciones y el error
            X_with_bias = np.column_stack((np.ones((m, 1)), X))
            y_pred = np.dot(X_with_bias, theta)
            error = y_pred - y
            
            # Actualiza los parámetros utilizando el gradiente
            theta -= (learning_rate / m) * np.dot(X_with_bias.T, error)
    
    if tipo == 'simple':
        return theta0, theta1
    elif tipo == 'multiple':
        return theta

In [ ]:
# Función de entrenamiento utilizando descenso de gradiente estocástico
def train_linear_regression_stochastic(X, y, learning_rate, num_iterations):
    # Inicializa los parámetros del modelo
    theta0 = 0.0
    theta1 = 0.0
    m = len(X)

    # Descenso de gradiente estocástico
    for _ in range(num_iterations):
        # Selecciona una muestra aleatoria
        random_index = np.random.randint(0, m)
        X_sample = X[random_index]
        y_sample = y[random_index]

        # Calcula la predicción y el error para la muestra seleccionada
        y_pred = theta0 + theta1 * X_sample
        error = y_pred - y_sample

        # Actualiza los parámetros utilizando el gradiente de la muestra seleccionada
        theta0 -= learning_rate * error
        theta1 -= learning_rate * error * X_sample

    return theta0, theta1

In [ ]:
  # Función para hacer predicciones
  def predict(X, theta = '', theta0 = '', theta1 = '', tipo_regresion='simple'):
    if tipo_regresion == 'simple':
      return theta0 + theta1 * X
    elif tipo_regresion == 'multiple':
      X_with_bias = np.column_stack([np.ones((len(X), 1)), X])  # Añade una columna de unos para el término de sesgo
      return np.dot(X_with_bias, theta)

In [ ]:
# Cálculo de las métricas de evaluación
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def r_squared(y_true, y_pred):
    ssr = np.sum((y_true - y_pred) ** 2)
    sst = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ssr / sst)

def adjusted_r_squared(y_true, y_pred, num_features):
    r2 = r_squared(y_true, y_pred)
    n = len(y_true)
    return 1 - ((1 - r2) * (n - 1) / (n - num_features - 1))

In [ ]:
# Funcion de regresion lineal simple
def regresion_lineal_simple(data, atributo, tipo_entrenamiento, tipo_regresion='simple'):
  # División de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
  train_size = int(0.8 * len(data))
  train_data = data[:train_size]
  test_data = data[train_size:]

  # Extrae las características y las etiquetas de entrenamiento y prueba
  train_X = train_data[atributo].values
  train_y = train_data['quality'].values
  test_X = test_data[atributo].values
  test_y = test_data['quality'].values
  
  if tipo_regresion == 'multiple':
    # Normalización de los datos de entrenamiento
    scaler = MinMaxScaler()
    train_X = scaler.fit_transform(train_X)

    # Normalización de los datos de prueba
    test_X = scaler.transform(test_X)

  # Hiperparámetros del modelo
  learning_rate = 0.01
  num_iterations = 1000

  # Entrena el modelo utilizando los datos de entrenamiento
  if tipo_entrenamiento == 'batch' and tipo_regresion == 'simple':
    theta0, theta1 = train_linear_regression(train_X, train_y, learning_rate, num_iterations)
  elif tipo_entrenamiento == 'batch' and tipo_regresion == 'multiple':
    theta = train_linear_regression(train_X, train_y, learning_rate, num_iterations, tipo='multiple')
  elif tipo_entrenamiento == 'stochastic':
    theta0, theta1 = train_linear_regression_stochastic(train_X, train_y, learning_rate, num_iterations)
    
  # Realiza predicciones en los conjuntos de entrenamiento y prueba
  if tipo_regresion == 'simple':
    train_predictions = predict(train_X, theta0=theta0, theta1=theta1)
    test_predictions = predict(test_X, theta0=theta0, theta1=theta1)
  elif tipo_regresion == 'multiple':
    train_predictions = predict(train_X, theta=theta, tipo_regresion=tipo_regresion)
    test_predictions = predict(test_X, theta=theta, tipo_regresion=tipo_regresion)

  # Calcular métricas de evaluación en los conjuntos de entrenamiento y prueba
  if tipo_regresion == 'simple':
    num_features = 1
  elif tipo_regresion == 'multiple':
    num_features = 3
    
  train_mse = mse(train_y, train_predictions)
  train_rmse = rmse(train_y, train_predictions)
  train_mae = mae(train_y, train_predictions)
  train_r2 = r_squared(train_y, train_predictions)
  train_adjusted_r2 = adjusted_r_squared(train_y, train_predictions, num_features)

  test_mse = mse(test_y, test_predictions)
  test_rmse = rmse(test_y, test_predictions)
  test_mae = mae(test_y, test_predictions)
  test_r2 = r_squared(test_y, test_predictions)
  test_adjusted_r2 = adjusted_r_squared(test_y, test_predictions, num_features)

  # Imprimir las métricas de evaluación
  print("Métricas de evaluación (conjunto de entrenamiento):")
  print("MSE:", train_mse)
  print("RMSE:", train_rmse)
  print("MAE:", train_mae)
  print("R^2:", train_r2)
  print("R^2 ajustado:", train_adjusted_r2)
  print("--------------------------------------------")
  print("Métricas de evaluación (conjunto de prueba):")
  print("MSE:", test_mse)
  print("RMSE:", test_rmse)
  print("MAE:", test_mae)
  print("R^2:", test_r2)
  print("R^2 ajustado:", test_adjusted_r2)
  
  if tipo_regresion == 'simple':
    # Gráfico de la regresión
    plt.scatter(train_X, train_y, color='blue', label='Training Data')
    #plt.scatter(test_X, test_y, color='red', label='Test Data')
    plt.plot(train_X, train_predictions, color='green', label='Regression Line')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Regresión lineal simple')
    plt.legend()
    plt.show()
  elif tipo_regresion == 'multiple':
    # plt.scatter(train_X[:, 0], train_y, color='blue', label='Training Data')
    # #plt.scatter(test_X[:, 0], test_y, color='red', label='Test Data')
    # plt.plot(train_X[:, 0], train_predictions, color='green', label='Regression Line')
    # plt.xlabel('X')
    # plt.ylabel('Y')
    # plt.title('Regresión lineal con múltiples variables')
    # plt.legend()
    # plt.show()
    # Grafico de los datos de entrenamiento en 3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(train_X[:, 0], train_X[:, 1], train_X[:, 2], c=train_y, cmap='viridis')
    ax.set_xlabel('Alcohol')
    ax.set_ylabel('Fixed Acidity')
    ax.set_zlabel('Volatile Acidity')
    ax.set_title('Datos de entrenamiento')
    plt.show()
    

## Regresión lineal simple usando batch gradient descent

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

### Para el Red Wine

#### Para el atributo Alcohol

In [ ]:
regresion_lineal_simple(redWine, 'alcohol', 'batch')

In [ ]:
# lista de atributos
atributos = ['alcohol', 'volatile acidity', 'pH']
regresion_lineal_simple(redWine, atributos, 'batch', 'multiple')

#### Para el atributo Acidez Volatil

In [ ]:
regresion_lineal_simple(redWine, 'volatile acidity', 'batch')

#### Para el atributo Sufates

In [ ]:
regresion_lineal_simple(redWine, 'sulphates', 'batch')

### Para el White Wine

#### Para el atributo Alcohol

#### Para el atributo Acidez Volatil

#### Para el atributo Sufates

## Regresion lineal simple usando stochastic gradient descent

### Para el White Wine

#### Para el atributo Alcohol

In [ ]:
regresion_lineal_simple(redWine, 'alcohol', 'stochastic')

#### Para el atributo Acidez Volatil

#### Para el atributo Density

### Para el Red Wine

#### Para el atributo Alcohol

#### Para el atributo Acidez Volatil

#### Para el atributo Density

## Analisis de los resultados para White Wine

## Analisis de los resultados para Red Wine

## Regresion Linear Multiple Batch Gradient Descent

## Regresion Linear Multiple Estocastica

# OTRAS WEAS